# Tasks - 

1. Normalize case
2. Tokenize (using word_tokenize from NLTK)
3. POS tagging using the NLTK pos tagger
4. For the topic model, we would want to include only nouns
 - First, find out all the POS tags that correspond to nouns
 - Limit the data to only terms with these tags
5. Lemmatize (you want different forms of the terms to be treated as one, don't worry about providing POS tag to lemmatizer for now)
6. Remove stop words and punctuation (if there are any at all after the POS tagging)
7. Create a topic model using LDA on the cleaned up data with 12 topics
 - choose the topic model parameters carefully
 - what is the perplexity of the model?
 - what is the coherence of the model?
8. Analyze the topics, which pairs of topics can be combined?
9. Create topic model using LDA with what you think is the optimal number of topics
 - choose the topic model parameters carefully
 - is the perplexity better now?
 - is the coherence better now?
10. The business finally needs to be able to interpret the topics
 - name each of the identified topics
 - create a table with the topic name and the top 10 terms in each to present to business

#### Some code to get you started

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Importing the usual utilities
import numpy as np, pandas as pd
import re, random, os, string

from pprint import pprint #pretty print
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#### Reading in the reviews file

In [3]:
reviews0 = pd.read_csv("K8 Reviews v0.2.csv")
reviews0.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


## Task 1. Normalize case

In [4]:
reviews_lower = [sent.lower() for sent in reviews0.review.values]
reviews_lower[0]

'good but need updates and improvements'

## Task 2. Tokenize (using word_tokenize from NLTK)

In [7]:
reviews_token = [word_tokenize(sent) for sent in reviews_lower]
reviews_token[0]

['good', 'but', 'need', 'updates', 'and', 'improvements']

## Task 3. POS tagging using the NLTK pos tagger

Check out the POS tagging chapter from the NLTK book

http://www.nltk.org/book/ch05.html

Examples - 

In [5]:
import nltk

In [8]:
nltk.pos_tag(reviews_token[0])

[('good', 'JJ'),
 ('but', 'CC'),
 ('need', 'VBP'),
 ('updates', 'NNS'),
 ('and', 'CC'),
 ('improvements', 'NNS')]

In [9]:
sent = "I like to move it".split()
sent_tagged = nltk.pos_tag(sent)

In [10]:
sent_tagged

[('I', 'PRP'), ('like', 'VBP'), ('to', 'TO'), ('move', 'VB'), ('it', 'PRP')]

In [11]:
reviews_tagged = [nltk.pos_tag(tokens) for tokens in reviews_token]
reviews_tagged[0]

[('good', 'JJ'),
 ('but', 'CC'),
 ('need', 'VBP'),
 ('updates', 'NNS'),
 ('and', 'CC'),
 ('improvements', 'NNS')]

## Task 4. For the topic model, we would want to include only nouns
 - First, find out all the POS tags that correspond to nouns
 - Limit the data to only terms with these tags


In [12]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

You see that for each term, the POS taggger returns a tuple. The first element of the tuple being the term, the second being the tag.

In [13]:
tagged_tuple = nltk.pos_tag(['great'])
tagged_tuple[0]

('great', 'JJ')

In [14]:
pprint(tagged_tuple[0][0])
pprint(tagged_tuple[0][1])

'great'
'JJ'


In [15]:
reviews_noun=[]
for sent in reviews_tagged:
    reviews_noun.append([token for token in sent if re.search("NN.*", token[1])])
reviews_noun[0]

[('updates', 'NNS'), ('improvements', 'NNS')]

You'll need to extract the tag from the resulting tuple, of course and then limit to the desired tags

## Task  5. Lemmatize
 - you want different forms of the terms to be treated as one
 - don't worry about providing POS tag to lemmatizer for now

In [17]:
lemm = WordNetLemmatizer()
reviews_lemm=[]
for sent in reviews_noun:
    reviews_lemm.append([lemm.lemmatize(word[0]) for word in sent])

In [18]:
reviews_lemm[0]

['update', 'improvement']

## Task  6. Remove stop words and punctuation (if there are any at all after the POS tagging)

Use NLTK standard stop word list and the punctuations

In [20]:
from string import punctuation
from nltk.corpus import stopwords
stop_nltk = stopwords.words("english")

In [21]:
stop_updated = stop_nltk + list(punctuation) + ["..."] + [".."]
reviews_sw_removed=[]
for sent in reviews_lemm:
    reviews_sw_removed.append([term for term in sent if term not in stop_updated])

In [22]:
reviews_sw_removed[1]

['mobile',
 'battery',
 'hell',
 'backup',
 'hour',
 'us',
 'idle',
 'discharged.this',
 'lie',
 'amazon',
 'lenove',
 'battery',
 'charger',
 'hour']

## Task 7. Create a topic model using LDA on the cleaned up data with 12 topics
 - what is the coherence of the model?
 
 Use gensim for this task

In [34]:
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models import ldamodel

In [35]:
id2word = corpora.Dictionary(reviews_sw_removed)
texts = reviews_sw_removed
corpus = [id2word.doc2bow(text) for text in texts]

In [36]:
print(corpus[200])

[(426, 1), (427, 1), (428, 1), (429, 1)]


In [41]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, 
                                           random_state=42,
                                           passes=10,
                                           per_word_topics=True)

In [40]:
pprint(lda_model.print_topics())

[(0,
  '0.212*"mobile" + 0.071*"time" + 0.071*"charger" + 0.053*"heat" + '
  '0.043*"charge" + 0.033*"hour" + 0.032*"turbo" + 0.027*"hr" + '
  '0.027*"charging" + 0.025*"bit"'),
 (1,
  '0.083*"feature" + 0.063*"call" + 0.031*"handset" + 0.029*"music" + '
  '0.022*"sensor" + 0.021*"ram" + 0.021*"sound" + 0.020*"power" + '
  '0.020*"speaker" + 0.019*"headphone"'),
 (2,
  '0.047*"glass" + 0.029*"gallery" + 0.027*"app" + 0.026*"system" + '
  '0.026*"buy" + 0.026*"internet" + 0.024*"gorilla" + 0.023*"screen" + '
  '0.021*"photo" + 0.020*"video"'),
 (3,
  '0.053*"hai" + 0.042*"h" + 0.023*"ho" + 0.015*"plz" + 0.015*"hi" + '
  '0.012*"fast" + 0.012*"bhi" + 0.012*"k" + 0.009*"ye" + 0.009*"ko"'),
 (4,
  '0.215*"battery" + 0.095*"camera" + 0.066*"performance" + 0.066*"problem" + '
  '0.044*"backup" + 0.044*"heating" + 0.029*"issue" + 0.025*"life" + '
  '0.023*"day" + 0.016*"mode"'),
 (5,
  '0.162*"money" + 0.083*"value" + 0.059*"work" + 0.049*"volta" + '
  '0.039*"worth" + 0.029*"killer" + 0.028*

In [43]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews_sw_removed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5308373844522803


## Task 8. Analyze the topics, which pairs of topics can be combined?
 - you can assume that if a pair of topics has very similar top terms, they are very close and can be combined

### Looking at the topics and each terms following can be combined -

** Topic 2 and 5 possibly talks about 'pricing'  
 Topic 4, 6 and 10 closely talks about 'battery related issues'  
 Topic 3 and 11 vaguely talks about 'performance'**

## Task 9. Create topic model using LDA with what you think is the optimal number of topics

 - is the coherence better now?

In [46]:
# Build LDA model
lda_model8 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=42,
                                           passes=10,
                                           per_word_topics=True)

Printing the coherence of the model

In [45]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model8, texts=reviews_sw_removed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5830478979190767


## Task 10. The business finally needs to be able to interpret the topics
 - name each of the identified topics
 - create a table with the topic name and the top 10 terms in each to present to business

In [47]:
x = lda_model8.show_topics(formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

In [48]:
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

0::['mobile', 'charger', 'waste', 'turbo', 'money', 'superb', 'box', 'ok', 'piece', 'awesome']
1::['product', 'note', 'k8', 'lenovo', 'service', 'handset', 'month', 'issue', 'experience', 'amazon']
2::['phone', 'screen', 'call', 'time', 'option', 'problem', 'day', 'app', 'replacement', 'note']
3::['hai', 'h', 'speed', 'ho', 'set', 'plz', 'hi', 'bhi', 'dont', 'future']
4::['battery', 'phone', 'problem', 'camera', 'issue', 'performance', 'backup', 'heating', 'day', 'hour']
5::['amazon', 'delivery', 'customer', 'return', 'money', 'system', 'product', 'star', 'service', 'time']
6::['camera', 'quality', 'network', 'issue', 'speaker', 'sound', 'sim', 'money', 'mode', 'value']
7::['phone', 'price', 'camera', 'feature', 'range', 'processor', 'performance', 'budget', 'ram', 'android']



In [49]:
#possible topics from terms present

#Topic1 = product accessories
#Topic2 = amazon
#Topic3 = pricing
#Topic4 = phone performance
#Topic5 = battery related issues
#Topic6 = camera quality
#Topic7 = sound features
#Topic8 = overall general phone features 